In [ ]:
import os
import torch 
import numpy as np
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

from src.utils import registry
import src.utils as utils
from S4Models import S4LMModel

In [ ]:
with initialize(version_base=None, config_path="configs"):
    
    config=compose(config_name="config_wiki_lm.yaml")
    print(OmegaConf.to_yaml(config))
    OmegaConf.set_struct(config, False)

In [ ]:

model = S4LMModel(config)
model.to('cpu')
state_dict = torch.load('artifacts/checkpoints:v144/pytorch_model.bin',map_location=torch.device('cpu'))
model.load_state_dict(state_dict,strict=True)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('google/long-t5-tglobal-base')
text = """Benedict Joseph Fenwick was born on September 3, 1782, at Beaverdam Manor in Leonardtown, Maryland,[1] to George Fenwick II, a planter and surveyor, and Margaret Fenwick née Medley.[2] His paternal ancestors hailed from Northumberland in North East England. Benedict's great-great-great grandfather, Cuthbert Fenwick, emigrated to America in the 1633 expedition of the Ark and the Dove, and was one of the original Catholic settlers of the British Province of Maryland.[3] Benedict's older brother was Enoch Fenwick, who would also become a prominent Jesuit,[4] and his cousin was Edward Fenwick, who would become a Dominican and Bishop of Cincinnati."""
prompt = tokenizer(text,add_special_tokens=False,return_tensors='pt')['input_ids']

In [ ]:
model.eval()
#model.to('cuda')
for module in model.modules():
    if hasattr(module, '_step_mode'): 
        module._step_mode = 'linear'
for module in model.modules():
    if hasattr(module, 'setup_step'): 
        module.setup_step()


In [ ]:
model.to('cuda')
model._reset_state(torch.tensor(prompt[0]).unsqueeze(0).unsqueeze(1),device='cuda')
output = []
with torch.no_grad():
    for word in tqdm(prompt.squeeze()):
        pred = model.generate(word.unsqueeze(0).unsqueeze(1).to('cuda'))
states = model._state

In [ ]:
output = [tokenizer.convert_ids_to_tokens(torch.argmax(pred[0]).detach().cpu().unsqueeze(0).numpy())]
model._state = states
top_p = 0.90
next_word = torch.argmax(pred[0]).detach().cpu().unsqueeze(0).numpy()
with torch.no_grad():
    for word in range(1800):
        y_t = model.generate(torch.tensor(next_word).unsqueeze(0).to('cpu'))
        probs = F.softmax(y_t, dim=-1)

        # Optional: nucleus sampling
        if top_p < 1.0:
            sorted_probs = probs.sort(dim=-1, descending=True)
            csum_probs = sorted_probs.values.cumsum(dim=-1) > top_p
            csum_probs[..., 1:] = csum_probs[..., :-1].clone()
            csum_probs[..., 0] = 0
            indices_to_remove = torch.zeros_like(csum_probs)
            indices_to_remove[torch.arange(sorted_probs.indices.shape[0])[:, None].repeat(1, sorted_probs.indices.shape[1]).flatten(), sorted_probs.indices.flatten()] = csum_probs.flatten()
            y_t = y_t + indices_to_remove.int() * (-1e20)
        next_word = Categorical(logits=y_t/0.9).sample()
        output += tokenizer.convert_ids_to_tokens([next_word])
        print(output[-1])